In [171]:
%ls

Specification.pdf         model.h5                  requirements.txt
Untitled.ipynb            processed.cleveland.data  venv/


In [214]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [192]:
df = pd.read_csv('processed.cleveland.data')
# df['thal'] = df['thal'].astype(int)
df = df.replace('?', np.nan).dropna()
df['ca'] = df['ca'].astype(float)
df['thal'] = df['thal'].astype(float)
df['thal'] = df['thal'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 301
Data columns (total 14 columns):
age         297 non-null float64
sex         297 non-null float64
cp          297 non-null float64
trestbps    297 non-null float64
chol        297 non-null float64
fbs         297 non-null float64
restecg     297 non-null float64
thalach     297 non-null float64
exang       297 non-null float64
oldpeak     297 non-null float64
slope       297 non-null float64
ca          297 non-null float64
thal        297 non-null int64
target      297 non-null int64
dtypes: float64(12), int64(2)
memory usage: 34.8 KB


In [193]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3,0


In [194]:
target = df.pop('thal')

In [201]:
target.unique()

array([6, 3, 7])

In [202]:
target[target == 3] = 0
target[target == 6] = 1
target[target == 7] = 2

In [203]:
cat_target = to_categorical(target, num_classes=3)
cat_target

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0

In [204]:
df = df.drop('target', axis=1)

In [205]:
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation=tf.nn.relu, input_shape=[len(df.keys())]),
        layers.Dropout(0.5),
        layers.Dense(128, activation=tf.nn.relu),
        layers.Dense(3, activation=tf.nn.softmax),
    ])
    
    optimiser = tf.keras.optimizers.RMSprop(0.001)
#     optimiser = tf.keras.optimizers.Adam(0.001)
    
    model.compile(loss='categorical_crossentropy',
                optimizer=optimiser,
                metrics=['mean_squared_error', 'accuracy'])
    
    
    return model

In [206]:
model = build_model()

In [207]:
EPOCHS = 10000

history = model.fit(df, cat_target, epochs=EPOCHS, validation_split=0.2, verbose=0)

In [208]:
hist = pd.DataFrame(history.history)
hist

,loss,mean_squared_error,acc,val_loss,val_mean_squared_error,val_acc
0,6.806302,0.300643,0.540084,7.253143,0.300000,0.550000
1,7.119438,0.300644,0.548523,7.253144,0.300000,0.550000
2,7.346628,0.307362,0.535865,7.253143,0.300000,0.550000
3,6.878185,0.310674,0.531646,3.803544,0.317818,0.466667
4,6.743667,0.304236,0.535865,6.208878,0.290849,0.566667
5,6.432106,0.314111,0.518987,7.530315,0.432732,0.333333
6,7.074227,0.328336,0.502110,6.478799,0.298680,0.550000
7,6.242369,0.309168,0.514768,7.082781,0.299953,0.550000
8,6.941294,0.311012,0.527426,7.157105,0.299999,0.550000
9,7.068628,0.323387,0.510549,5.957770,0.287397,0.566667


In [209]:
model.predict(df[:10]).argmax(axis=1)

array([2, 0, 2, 0, 0, 0, 0, 0, 2, 0])

In [210]:
cat_target[:10].argmax(axis=1)

array([1, 0, 2, 0, 0, 0, 0, 0, 2, 2])

In [211]:
model.predict(df[-10:]).argmax(axis=1)

array([2, 2, 0, 0, 2, 0, 2, 2, 2, 0])

In [212]:
cat_target[-10:].argmax(axis=1)

array([1, 2, 0, 0, 1, 2, 2, 2, 2, 0])

In [213]:
model.save('model.h5')

In [215]:
model2 = load_model('model.h5')

In [217]:
model2.predict(df[:10]).argmax(axis=1)

array([2, 0, 2, 0, 0, 0, 0, 0, 2, 0])